In [7]:
import os
from dotenv import find_dotenv, load_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI-API-KEY')
langchain_api_key = os.getenv('LANGCHAIN-API-KEY')

In [41]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

In [45]:
loader = WebBaseLoader(
    web_paths=["https://www.fcm.dk/billetter/", 'https://www.fcm.dk/saesonkort/']
) #Consider including argument that removes irrelevant text

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [59]:
print(f"Number of splits: {len(splits)}")
print(splits[-5:])  # Print the first few splits to inspect their content


Number of splits: 111
[Document(metadata={'source': 'https://www.fcm.dk/saesonkort/', 'title': 'Sæsonkort', 'language': 'da-DK'}, page_content='Kan jeg skifte plads på mit sæsonkort?\n\t\t\t\t\t\t\t\t\n\n\n\n\n\n\n\n\n\nJa, det kan du, men det kræver, at du skal have et nyt sæsonkort til din nye plads. Det koster et gebyr på 100 kr. at printe et sæsonkort. Skriv til os på billetsalg@fcm.dk med nummeret på dit nuværende sæsonkort og pladsen på dit nye sæsonkort, så kan vi gøre det for dig. Du skal betale differencen mellem pladserne samt gebyret på 100 kr.'), Document(metadata={'source': 'https://www.fcm.dk/saesonkort/', 'title': 'Sæsonkort', 'language': 'da-DK'}, page_content='Kan jeg sidde/stå på en anden plads/tribune til en enkelt kamp?'), Document(metadata={'source': 'https://www.fcm.dk/saesonkort/', 'title': 'Sæsonkort', 'language': 'da-DK'}, page_content='Ja, det kan du, men så skal du have en billet til den enkelte kamp. Skriv til os på billetsalg@fcm.dk med nummeret på dit sæso

In [51]:
#Define vectorstore and retriever
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=openai_api_key))
retriever = vectorstore.as_retriever(k=5)

In [52]:
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "Du er en assistent der skal svare på spørgsmål fra fans af fodbolklubben FC Midtjylland. "
    "Brug følgende kontekst til at besvare spørgsmålet. "
    "Hvis du ikke kender svaret, henvis til mailen billetsalg@fcm.dk. "
    "Brug maks fire sætninger og svar præcist. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [53]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [54]:
response = rag_chain.invoke({"input": "Hvad er billetflex?"})
response["answer"]

'BilletFlex er en løsning for dem, der ønsker adgang til alle kampe på MCH Arena, men ikke kan deltage hver gang. Med BilletFlex skal du aktivt trække billetter fra kamp til kamp, så du ikke optager en plads til de kampe, du ikke deltager i. Hvis du kommer til mindst 65% af kampene i en sæson, vil du blive tilbudt et sæsonkort med fast plads i den efterfølgende sæson.'